<a href="https://colab.research.google.com/github/reddy26lc/Exploratory-Data-Analysis/blob/master/Creating_CNN_Model_and_optimizing_it_using_Keras_Tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 5.5MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=ba592fe5f4503b63eac938e604d414dc0d0b76e941ab3503d05792584f990a56
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=02fca29e6591f50786c6072ab1d50bfdefd61ed49d44d3a93479a5381bea3ec7
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [4]:
print(tf.__version__)

2.3.0


In [5]:
#Importing the dataset from the tensorflow library itself

fashion_mnist = keras.datasets.fashion_mnist

In [6]:
(train_images, train_labels),(test_images,test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [7]:
#Scale down the images between 0 to 1. All the pixels are converted to between 0 and 1

train_images = train_images/255.0
test_images = test_images/255.0

In [8]:
train_images[0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [9]:
train_images.shape

(60000, 28, 28)

In [10]:
train_images[0].shape #the first images has 28 by 28 pixels. All images have that

(28, 28)

In [11]:
#We need to reshape it since we have 28 by 28 pixels
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [12]:
#We wil create a function called build model and inside it hp is the hyperparameter
#kernel size is filter size. Keras tuner will find which conv2D layer can be used and what filter size
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter',min_value=32,max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel',values=[3,5]),
        activation ='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter',min_value=32,max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel',values=[3,5]),
        activation ='relu',
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units',min_value=32,max_value=128,step=16),
        activation='relu'
    ),
    keras.layers.Dense(10,activation='softmax')
  ]) #10 output nodes

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[1e-2,1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model
  

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
#this step will help us to search which convultion layer should be used, whats the filter size, learning rate etc

tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='output',project_name='MNIST_Fashion')

In [15]:
#When we give max_trials we will only run for 2-3 epochs and find the best hyperparameters, later we can run for many epochs
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3705 - accuracy: 0.8668 - val_loss: 0.2698 - val_accuracy: 0.9022
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2271 - accuracy: 0.9169 - val_loss: 0.2712 - val_accuracy: 0.8977
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1565 - accuracy: 0.9419 - val_loss: 0.2355 - val_accuracy: 0.9135


Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3508 - accuracy: 0.0975 - val_loss: 2.3025 - val_accuracy: 0.1008
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3041 - accuracy: 0.1004 - val_loss: 2.3029 - val_accuracy: 0.0973
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3039 - accuracy: 0.0996 - val_loss: 2.3063 - val_accuracy: 0.1027


Epoch 1/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.3889 - accuracy: 0.8610 - val_loss: 0.2887 - val_accuracy: 0.8947
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2421 - accuracy: 0.9099 - val_loss: 0.2494 - val_accuracy: 0.9072
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1840 - accuracy: 0.9320 - val_loss: 0.2550 - val_accuracy: 0.9060


Epoch 1/3
1688/1688 [==============================] - 8s 4ms/step - loss: 0.3862 - accuracy: 0.8620 - val_loss: 0.2987 - val_accuracy: 0.8885
Epoch 2/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.2407 - accuracy: 0.9113 - val_loss: 0.2580 - val_accuracy: 0.9027
Epoch 3/3
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1793 - accuracy: 0.9333 - val_loss: 0.2322 - val_accuracy: 0.9140


Epoch 1/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.3854 - accuracy: 0.8608 - val_loss: 0.2922 - val_accuracy: 0.8932
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2443 - accuracy: 0.9088 - val_loss: 0.2581 - val_accuracy: 0.9035
Epoch 3/3
1688/1688 [==============================] - 9s 6ms/step - loss: 0.1809 - accuracy: 0.9320 - val_loss: 0.2614 - val_accuracy: 0.9060


INFO:tensorflow:Oracle triggered exit


In [16]:
#to get the best model. It will give a list so to pick the first model we will index it with [0] 
model = tuner_search.get_best_models(num_models=1)[0]

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                2211920   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 2,241,066
Trainable params: 2,241,066
Non-trainable params: 0
_________________________________________________________________


In [19]:
#So considering the above best model, we need to retrain all the images. 
model.fit(train_images, train_labels, epochs=10, validation_split=0.1,initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1318 - accuracy: 0.9503 - val_loss: 0.2782 - val_accuracy: 0.9125
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0931 - accuracy: 0.9656 - val_loss: 0.3093 - val_accuracy: 0.9067
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0669 - accuracy: 0.9755 - val_loss: 0.3349 - val_accuracy: 0.9062
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0466 - accuracy: 0.9829 - val_loss: 0.3768 - val_accuracy: 0.9120
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0337 - accuracy: 0.9876 - val_loss: 0.4778 - val_accuracy: 0.9088
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0298 - accuracy: 0.9891 - val_loss: 0.5654 - val_accuracy: 0.9048
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0240 - accuracy: 0.9913 - val_loss: 0.4885 - val_accuracy